In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import requests

service = Service(executable_path="./chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get("https://www.google.com")
driver.maximize_window()
input = driver.find_element(By.CLASS_NAME,"gLFyf")
input.send_keys("layoffs.fyi" + Keys.ENTER)
link = driver.find_element(By.PARTIAL_LINK_TEXT, "layoff")
link.click()

data = BeautifulSoup(requests.get("https://layoffs.fyi/").content,"html.parser")
d = data.find_all("iframe",attrs={"class":"airtable-embed"})
links = [link["src"] for link in d]
driver.get(links[0])

company_list = []
layoff_list = []
location_list = []
date_list = []
list_country = []
list_industry =[]
  
i = 0
while(i < 200):
    verticalScrollBar = driver.find_element(By.CSS_SELECTOR, ".antiscroll-scrollbar.antiscroll-scrollbar-vertical.antiscroll-scrollbar-shown")
    ActionChains(driver).drag_and_drop_by_offset(verticalScrollBar, 0, 2).perform()
    
    #adjusting right panel for dates,laid off , location, ...
    rigth_panel =  driver.find_element(By.CSS_SELECTOR, ".rightPaneWrapper")
    right_panel_soup = BeautifulSoup(rigth_panel.get_attribute("outerHTML"),"html.parser")
    
    # addding country to list 
    countries = right_panel_soup.find_all("div",attrs={"data-columnid":"fldnR6AcR7HjRtwrG"})
    list_country += [country.text for country in countries]
    
    # adding dates to date_list
    dates = right_panel_soup.find_all("div",attrs={"class":"truncate css-10jy3hn"})
    date_list += [date.text for date in dates]
    
    #adding industry to list
    industries = right_panel_soup.find_all("div",attrs={"data-columnid":"fldMvtAoxHCri39uZ"})
    list_industry += [industry.text for industry in industries]
    # locations    
    locations = right_panel_soup.find_all("div",attrs={"class":"baymax flex flex-auto overflow-hidden fit p-half"})
    location_list += [location.text for location in locations]
    
    # adding laid off to layoff_list
    layoff = right_panel_soup.find_all("div",attrs={"data-columnid":"flduZSpdFqkB4eeEh"})
    layoff_list += [lay.text for lay in layoff]
    
    # adjust left panel's data to add company names to list 
    left_panel= driver.find_element(By.CSS_SELECTOR, ".leftPaneWrapper")
    left_panel_soup = BeautifulSoup(left_panel.get_attribute("outerHTML"),"html.parser")
    company_names = left_panel_soup.find_all("div",attrs={"class":"line-height-4 overflow-hidden truncate-block-1-line truncate"})
    company_list += [company.text for company in company_names]

    i = i + 1
        
time.sleep(5)

driver.quit()

In [ ]:
layoff_list = [x for x in layoff_list if x != '\n\n\n# Laid Off\n\n\n\n\n\n\n\n\n\n\n\n']
#print(layoff_list)
list_industry = [x for x in list_industry if x != '\n\n\nIndustry\n\n\n\n\n\n\n\n\n\n\n\n']
list_country = [x for x in list_country if x != '\n\n\nCountry\n\n\n\n\n\n\n\n\n\n\n\n']
print(list_country)

In [ ]:
"""
import csv 
with open("Layoff.csv","w",newline="",encoding = "utf-8") as file:
    for a1,a2,a3,a4,a5,a6 in zip(company_list,layoff_list,location_list,date_list,list_country,list_industry):
        file.write(a1 + "," + a2 + "," + a3 + "," + a4 + ","+a5 + ","+a6 +"\n")
"""

In [ ]:
import sqlite3

connection = sqlite3.connect("layoff.db")
process = connection.cursor()
connection.commit()


In [ ]:
table = process.execute("Create table if not exists csv(company TEXT, layoff TEXT,location TEXT,date TEXT,country TEXT, industry TEXT)")
connection.commit()

In [ ]:
list_of_datalist = zip(company_list,layoff_list,location_list,date_list,list_country,list_industry)

for data in list_of_datalist:
    process.execute("Insert into csv values(?,?,?,?,?,?)",data)
    connection.commit()